In [1]:
import os
import numpy as np
import sys
import pandas as pd
from sqlalchemy import text
## shift directories, specify a different folder, stop execution of the script once it's running
import logging
## to create logs to tell us what's happening
from fun.connect import URL_maker, engineConnector
from fun.extraction import extractPostgresTable, extractPostgresTableSchema
from fun.csvchecks import testCols, typeCheck, timeCheck
from fun.load import loadToPostgresIncrement


In [2]:
#Set up log- in the folder called logs, create a mainlog folder (os make dirs is to only make it if it doens't exist already)
logFileName="logs/mainlog.log"
os.makedirs(os.path.dirname(logFileName), exist_ok=True)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(message)s',
    handlers=[
        logging.FileHandler(logFileName)
    ]
)

logging.info('\n Attempting new connection')

In [3]:
conURL=URL_maker('ADFTUv2.ini')
logging.info(f'Attempting connection at {conURL}')

In [4]:
engine = engineConnector(conURL)

In [5]:
## load original file into a dataframe, make sure N/As are read as strings

df = pd.read_csv('ECMC_portfolio_22_23.csv',na_values=[], keep_default_na=False)

df = df.replace('', np.nan)

# Display the first few rows to check if it loaded correctly
df.head()

,0.1. ECMC name\nPlease enter your Centre name for each row of trial data,"0.2. Your reference ID\nOPTIONAL\nThis is the ID that you use at your centre to identify the trial.\nThis will not be used by the ECMC Programme office, it is included to help you report.",0.3. ECMC Supported?\nPlease select Yes/No to indicate if the trial uses significan ECMC funded STAFF resources,1.1. Full trial title\nThis should be identical to the title on the trial protocol and not an abbreviated version,"1.2. Trial acronym\nAcronym, abbreviation or alternative trial name which can help users outside your Centre to find this trial. Not all studies have one","1.3. NIHR Portfolio/ CPMS ID\nIf the trial is supported by the NIHR CRN and is on the NIHR Portfolio/CPMS, please provide the ID. \nPlease leave blank if you don’t have this number","1.4. IRAS Project Number\nIf the Trial is still in set-up, and yet to be submitted through IRAS, please enter “TBC”.",1.5. REC reference number\nReference number assigned by the Research Ethics Committee (REC) accepting the application for review.,"1.6. Clinical Trial Register ID\nIf the trial is on any of the following clinical Trial Registers, please provide the register ID:\nEU Clinical Trials Register: EUDRACT ID\nClinicalTrials.gov: NCTXXXXXXX\nISRCTN: ISRCTNXXXXX",1.7. Trial Phase \nThe phase of the trial (as stated in current trial protocol).,...,"4.6. Date Site Confirmed By Sponsor\nDate of the first contract signature of all the organisations involved (i.e. sponsor, site, 3rd party) or Date on the email received from Sponsor providing the final statement of activity ready for final agreement.","4.7. Date Site Confirmed\nDate of the last contract signature of all the organisations involved (i.e. sponsor, site, 3rd party) or date of final written agreement of statement of activity (as applicable)","4.8. Date site Ready To Start Screening activites - ""Sponsor Green light/Site Open permission given to commence screening""","4.9. Date of First Patient Recruited \nThe date of consent to participate for an eligible participant\nWhere a participant consents to both screening and participation into a study and they then fail to pass screening, their consent date cannot be used.",5.1. SITE Patient recruitment from 1 April 2021 - 31 March 2022\nThis is the number of patients recruited between 1 April 2021 - 31 March 2022 at your site,5.2. SITE recruitment status\nWhat is the current status of a trial at your ECMC?,5.3. UK WIDE Patient Recruitment target \n\n(LEAD Site to complete),6.1. Site notes to the ECMC Programme Office,6.2 PO Notes to Sites,6.3 PO Additional Data - TRIAL INTERVENTIONS
0,Belfast,17039SS-SS,Yes,"A Phase I/IIA Study to Assess Safety, Tolerabi...",FAK-PD1,35657,200488,16/WS/0093,2015-003928-31;NCT02758587,Phase I/II,...,28/09/2017,11/10/2017,24/11/2017,26/03/2018,0,"Closed, Follow Up Complete",57,NaN,NaN,"defactinib (Anticancer, other);pembrolizumab (..."
1,Glasgow,SA172,Yes,"A Phase I/IIA Study to Assess Safety, Tolerabi...",FAK-PD1,35657,200488,16/WS/0093,2015-003928-31;NCT02758587,Phase I/II,...,24/04/2017,04/05/2017,23/05/2017,04/07/2017,0,"Closed, in Follow up",57,"2.8 Blood biomarkers, Archival tissue tumour b...",NaN,"defactinib (Anticancer, other);pembrolizumab (..."
2,Southampton,RHM CAN1310,Yes,"A Phase I/IIA Study to Assess Safety, Tolerabi...",FAK-PD1,35657,200488,16/WS/0093,2015-003928-31;NCT02758587,Phase I/II,...,08/09/2017,23/10/2017,12/12/2017,18/01/2018,0,"Closed, Follow Up Complete",57,NaN,NaN,"defactinib (Anticancer, other);pembrolizumab (..."
3,Oxford,13155,Yes,"A Phase 1, Open to recruitment-label, Dose-Fin...",CC-90009,35718,232667,17/SC/0531,2017-001535-39;NCT02848001,Phase I,...,05/10/2018,05/10/2018,13/11/2018,01/10/2018,0,"Closed, in Follow up",6,NaN,NaN,"CC-90009 (Anticancer, other)"
4,Cardiff,NaN,0,Phase 1b/II Trial of Checkpoint Inhibitor (Pem...,CORINTH,35838,228287,19/LO/0314,2017-002300-27,Phase I/II,...,10/09/2021,09/09/2021,10/09/2021,01/11/2021,1,Recruitment p

In [6]:
### Log starting tests

#Integrity checks
logging.info('Starting tests...')

In [7]:
### Check number of columns
try:
    expected_num_columns = 46
    actual_num_columns = len(df.columns)

    assert actual_num_columns == expected_num_columns, f"Expected {expected_num_columns} columns, but got {actual_num_columns}"
    
    logging.info('Number of Columns: OK')

except AssertionError as e:
    logging.error(f"ERROR: Number of columns mismatch: {e}")
    sys.exit(1)

except Exception as e:
    logging.error(f"ERROR: An unexpected error occurred: {e}")
    sys.exit(1)


In [8]:
###test specific names of columns
##Tests always in try loops
## devops principles (assert - sets a logical test)

try:
    expectedCols = ['0.1. ECMC name\nPlease enter your Centre name for each row of trial data',
       '0.2. Your reference ID\nOPTIONAL\nThis is the ID that you use at your centre to identify the trial.\nThis will not be used by the ECMC Programme office, it is included to help you report.',
       '0.3. ECMC Supported?\nPlease select Yes/No to indicate if the trial uses  significan ECMC funded STAFF resources  ',
       '1.1. Full trial title\nThis should be identical to the title on the trial protocol and not an abbreviated version',
       '1.2. Trial acronym\nAcronym, abbreviation or alternative trial name which can help users outside your Centre to find this trial. Not all studies have one',
       '1.3. NIHR Portfolio/ CPMS ID\nIf the trial is supported by the NIHR CRN and is on the NIHR Portfolio/CPMS, please provide the ID. \nPlease leave blank if you don’t have this number',
       '1.4. IRAS Project Number\nIf the Trial is still in set-up, and yet to be submitted through IRAS, please enter “TBC”.',
       '1.5. REC reference number\nReference number assigned by the Research Ethics Committee (REC) accepting the application for review.',
       '1.6. Clinical Trial Register ID\nIf the trial is on any of the following clinical Trial Registers, please provide the register ID:\nEU Clinical Trials Register: EUDRACT ID\nClinicalTrials.gov: NCTXXXXXXX\nISRCTN: ISRCTNXXXXX',
       '1.7. Trial Phase \nThe phase of the trial (as stated in current trial protocol). ',
       '1.8. Cancer Type/s \nPlease include ALL cancer type/s being investigated as part of the trial. \n\nMultiple select drop down\nNOTE: if you have selected a disease by error; delete all contents of the cell and re-enter the correct field options',
       '1.9. Trial Treatment category\nPlease enter the treatment category/ies relevant to the trial.\n* Combination study - Drug/radiotherapy/surgery/device\n* Immunotherapy\n* Advanced Therapies \n\nMultiple select drop down\nNOTE: if you have selected a disease by error, delete all contents of the cell and re-enter the correct field options',
       '1.10. Trial Sponsor', '1.11. Trial Funder/s',
       '1.12. Is this an INVESTIGATOR INITIATED STUDY?\nInvestigator  initiated studies (IIS) are clinical studies where funding has been awarded from a commercial organisation, but the protocol and study development are sponsored and managed by a non-commercial organisation ',
       '1.13. International/National trial \nIs your trial also planned/running internationally? If UK only, is it Multi-site or single site trial',
       '1.14. Patient Age group\nUsing the age criteria for patient eligibility, please select the relevant age range option ',
       '2.1. If relevant, is your site participating on the DOSE ESCALATION part of this trial?\n\n(N/A on paediatric spreadsheet)',
       '2.2. Is this trial investigating a FIRST IN CLASS DRUG?\nPlease indicate if one or more treatments being tested within the trial is classified as First-In-class?\nDefinition: First-in-class drugs are ones that use a new and unique mechanism of action for treating a medical condition. These products are often referred to as innovative and cited as offering new treatment options for patients \n\n(N/A on Paediatric spreadsheet)',
       '2.3. Is this a FIRST IN CHILD study\n',
       '2.4. Is this a PRECISION MEDICINE study?\nDefinition: Precision Medicine study definition: trials that stratifies patients based on genetic and/or other biomarker information (molecular stratification)',
       '2.5. Is this an ADAPTIVE DESIGN study\nThese are trials where potential adaptations of the study conduct are defined in the initial study protocol.  Examples of adaptive design studies: \n- studies which include the option of modifying key components \n- studies which include possible phase expansion from phase II into phase III\n- studies which allow for the possibility of adding or dropping treatment arms or determining response at interim.',
       '2.6. Is trial RANDOMISED?',
       '2.7. What PRE-SCREENING PROCESSES are in place for the trial that require pre-screening consent (e.g  requesting consent to access a patient\'s previous biopsies/samples)\nIf there is more than 1 processes, please separate each with a comma\nPlease enter "N/A" if there are no pre-screening activities',
       '2.8. Does the trial require RESOURCE-INTENSIVE SCREENING with multiple steps: e.g molecular screening, biopsy, imaging, other -(if other, please provide additional info in Section 6)\n\nMultiple select drop down\nNOTE: if you have selected a disease by error, delete all contents of the cell and re-enter the correct field options',
       '3.1. Number of patients screened at your site 1 April 2021 - 31 March 2022\nONLY Answer if trial uses RESOURCE-INTENSIVE SCREENING (field 2.8)',
       '3.2. TOTAL accumulative NUMBER OF EXPERIMENTAL ARMS at your Site (planned, open and closed)\nPlease report on the current total accumulative number of study EXPERIMENTAL arms that have closed, are open or planned at your Site since the start of the trial.\nExperimental arms as defined in the Trial Protocol. If there are molecularly defined disease cohorts under these arms, please add this information in Section 6 (far right column of the dataset)',
       '3.3. Number of SIMULTANEOUSLY OPEN EXPERIMENTAL ARMS in your site since trial open\nPlease report on the maximum number of study EXPERIMENTAL arms open simultaneously at your site any point in its lifetime\nExperimental arms as defined in the Trial Protocol. If there are molecularly defined disease cohorts under these arms, please add this information in Section 6 (far right column of the dataset)',
       '3.4. Are any researcher/clinician at Site part of the trial steering group\nPlease indicate if a researcher/clinician at your Site is/was part of the trial steering group.\nSteering committees are responsible for decision-making and are the highest level of oversight committee for a clinical trial. \nIf there is no Steering Committee for this trial, please indicate if a researcher/clinician at your site is a key decision maker for this trial',
       '3.5. Is your site the Lead site for the UK?\nIs the Chief Investigator* (not Principal Investigator) for this trial based at your site?',
       '3.6. Is the UK CI the GLOBAL CI for this trial \nOnly to be completed for international; multi-site studies\nIf the trial is also running internationally and the CI is based at your site, is the Chief Investigator the Global CI for the trial based\n\n(LEAD Site to complete)',
       '4.1. Date of HRA application\nIf your site was responsible for applying for HRA approval process,  please provide the date the trial application for this trial was submitted through the IRAS system.\n\n(LEAD Site to complete)',
       '4.2. Date Site Invited \nDate of the email from the Sponsor to the site which provides the protocol in the version to be submitted for regulatory review\nON NIHR Performance in initiating and Delivery of trials list',
       '4.3. Date Site Selected \nDate on the Sponsor email received by the site providing the minimum defined documents to enable site to commence arrangement and/or confirmation of local capacity and capability as applicable representing that the site has been selected to take part in the trial. ',
       '**NEW**\n4.4. Date of receipt of HRA document submission pack from sponsors \n(aka Document submission pack)\nDate of receipt of all necessary documentation required for the permissions and approvals process.',
       '4.5. HRA Approval Date \nDate of HRA Approval for trial as per HRA Approval Letter\n\n(LEAD Site to complete)',
       '4.6. Date Site Confirmed By Sponsor\nDate of the first contract signature of all the organisations involved (i.e. sponsor, site, 3rd party) or Date on the email received from Sponsor providing the final statement of activity ready for final agreement. ',
       '4.7. Date Site Confirmed\nDate of the last contract signature of all the organisations involved (i.e. sponsor, site, 3rd party) or date of final written agreement of statement of activity (as applicable)',
       '4.8. Date site Ready To Start Screening activites - "Sponsor Green light/Site Open permission given to commence screening" ',
       '4.9. Date of First Patient Recruited \nThe date of consent to participate for an eligible participant\nWhere a participant consents to both screening and participation into a study and they then fail to pass screening, their consent date cannot be used. ',
       '5.1. SITE Patient recruitment from 1 April 2021 - 31 March 2022\nThis is the number of patients recruited between 1 April 2021 - 31 March 2022 at your site',
       '5.2. SITE recruitment status\nWhat is the current status of a trial at your ECMC?',
       '5.3. UK WIDE Patient Recruitment target \n\n(LEAD Site to complete)',
       '6.1. Site notes to the ECMC Programme Office', '6.2 PO Notes to Sites',
       '6.3 PO Additional Data - TRIAL INTERVENTIONS']
    testCols(df,expectedCols)
    logging.info('Column Names: OK')

except AssertionError as e:
    logging.error(f"ERROR: Column name mismatch: {e}")
    sys.exit(1)

except Exception as e:
    logging.error(f"ERROR: An unexpected error occured: {e}")
    sys.exit(1)
   

In [9]:
# Rename specific columns
df.rename(columns={
    '0.1. ECMC name\nPlease enter your Centre name for each row of trial data': 'ecmc_name', 
    '1.1. Full trial title\nThis should be identical to the title on the trial protocol and not an abbreviated version': 'full_trial_title',
    '1.2. Trial acronym\nAcronym, abbreviation or alternative trial name which can help users outside your Centre to find this trial. Not all studies have one': 'trial_acronym',
    '1.3. NIHR Portfolio/ CPMS ID\nIf the trial is supported by the NIHR CRN and is on the NIHR Portfolio/CPMS, please provide the ID. \nPlease leave blank if you don’t have this number': 'cpms_id',
    '1.4. IRAS Project Number\nIf the Trial is still in set-up, and yet to be submitted through IRAS, please enter “TBC”.': 'iras_id',
    '1.5. REC reference number\nReference number assigned by the Research Ethics Committee (REC) accepting the application for review.': 'rec_reference_number',
    '1.13. International/National trial \nIs your trial also planned/running internationally? If UK only, is it Multi-site or single site trial': 'international_national_trial',
    '1.14. Patient Age group\nUsing the age criteria for patient eligibility, please select the relevant age range option ': 'patient_age_group',
    '2.1. If relevant, is your site participating on the DOSE ESCALATION part of this trial?\n\n(N/A on paediatric spreadsheet)': 'dose_escalation',
    '2.2. Is this trial investigating a FIRST IN CLASS DRUG?\nPlease indicate if one or more treatments being tested within the trial is classified as First-In-class?\nDefinition: First-in-class drugs are ones that use a new and unique mechanism of action for treating a medical condition. These products are often referred to as innovative and cited as offering new treatment options for patients \n\n(N/A on Paediatric spreadsheet)': 'first_in_class',
    '2.3. Is this a FIRST IN CHILD study\n': 'first_in_child',
    '2.4. Is this a PRECISION MEDICINE study?\nDefinition: Precision Medicine study definition: trials that stratifies patients based on genetic and/or other biomarker information (molecular stratification)': 'precision_medicine',
    '2.5. Is this an ADAPTIVE DESIGN study\nThese are trials where potential adaptations of the study conduct are defined in the initial study protocol.  Examples of adaptive design studies: \n- studies which include the option of modifying key components \n- studies which include possible phase expansion from phase II into phase III\n- studies which allow for the possibility of adding or dropping treatment arms or determining response at interim.': 'complex_innovative_design',
    '2.6. Is trial RANDOMISED?': 'randomised',
    '5.3. UK WIDE Patient Recruitment target \n\n(LEAD Site to complete)': 'uk_wide_patient_recruitment_target',
    '3.1. Number of patients screened at your site 1 April 2021 - 31 March 2022\nONLY Answer if trial uses RESOURCE-INTENSIVE SCREENING (field 2.8)': 'number_of_patients_screened',
    '3.2. TOTAL accumulative NUMBER OF EXPERIMENTAL ARMS at your Site (planned, open and closed)\nPlease report on the current total accumulative number of study EXPERIMENTAL arms that have closed, are open or planned at your Site since the start of the trial.\nExperimental arms as defined in the Trial Protocol. If there are molecularly defined disease cohorts under these arms, please add this information in Section 6 (far right column of the dataset)': 'total_accumulative_number_of_experimental_arms',
    '3.3. Number of SIMULTANEOUSLY OPEN EXPERIMENTAL ARMS in your site since trial open\nPlease report on the maximum number of study EXPERIMENTAL arms open simultaneously at your site any point in its lifetime\nExperimental arms as defined in the Trial Protocol. If there are molecularly defined disease cohorts under these arms, please add this information in Section 6 (far right column of the dataset)': 'number_of_simultaneously_open_experimental_arms',
    '3.5. Is your site the Lead site for the UK?\nIs the Chief Investigator* (not Principal Investigator) for this trial based at your site?': 'lead_site',
    '4.1. Date of HRA application\nIf your site was responsible for applying for HRA approval process,  please provide the date the trial application for this trial was submitted through the IRAS system.\n\n(LEAD Site to complete)': 'date_of_hra_application',
    '4.8. Date site Ready To Start Screening activites - "Sponsor Green light/Site Open permission given to commence screening" ': 'date_site_ready_to_start',
    '4.9. Date of First Patient Recruited \nThe date of consent to participate for an eligible participant\nWhere a participant consents to both screening and participation into a study and they then fail to pass screening, their consent date cannot be used. ': 'date_of_first_patient_recruited',
    '5.1. SITE Patient recruitment from 1 April 2021 - 31 March 2022\nThis is the number of patients recruited between 1 April 2021 - 31 March 2022 at your site': 'site_patient_recruitment',
    '5.2. SITE recruitment status\nWhat is the current status of a trial at your ECMC?': 'site_recruitment_status',
    '1.10. Trial Sponsor': 'sponsor',
    '1.8. Cancer Type/s \nPlease include ALL cancer type/s being investigated as part of the trial. \n\nMultiple select drop down\nNOTE: if you have selected a disease by error; delete all contents of the cell and re-enter the correct field options': 'cancer_type',
    '1.9. Trial Treatment category\nPlease enter the treatment category/ies relevant to the trial.\n* Combination study - Drug/radiotherapy/surgery/device\n* Immunotherapy\n* Advanced Therapies \n\nMultiple select drop down\nNOTE: if you have selected a disease by error, delete all contents of the cell and re-enter the correct field options': 'treatment_type',
    '1.7. Trial Phase \nThe phase of the trial (as stated in current trial protocol). ': 'trial_phase',
    '2.7. What PRE-SCREENING PROCESSES are in place for the trial that require pre-screening consent (e.g  requesting consent to access a patient\'s previous biopsies/samples)\nIf there is more than 1 processes, please separate each with a comma\nPlease enter "N/A" if there are no pre-screening activities': 'pre_screening_processes',
    '4.5. HRA Approval Date \nDate of HRA Approval for trial as per HRA Approval Letter\n\n(LEAD Site to complete)':'hra_approval_date',
    '1.12. Is this an INVESTIGATOR INITIATED STUDY?\nInvestigator  initiated studies (IIS) are clinical studies where funding has been awarded from a commercial organisation, but the protocol and study development are sponsored and managed by a non-commercial organisation ' : 'investigator_initiated_study'
    
}, inplace=True)

# Display the updated DataFrame
df.head()

,ecmc_name,"0.2. Your reference ID\nOPTIONAL\nThis is the ID that you use at your centre to identify the trial.\nThis will not be used by the ECMC Programme office, it is included to help you report.",0.3. ECMC Supported?\nPlease select Yes/No to indicate if the trial uses significan ECMC funded STAFF resources,full_trial_title,trial_acronym,cpms_id,iras_id,rec_reference_number,"1.6. Clinical Trial Register ID\nIf the trial is on any of the following clinical Trial Registers, please provide the register ID:\nEU Clinical Trials Register: EUDRACT ID\nClinicalTrials.gov: NCTXXXXXXX\nISRCTN: ISRCTNXXXXX",trial_phase,...,"4.6. Date Site Confirmed By Sponsor\nDate of the first contract signature of all the organisations involved (i.e. sponsor, site, 3rd party) or Date on the email received from Sponsor providing the final statement of activity ready for final agreement.","4.7. Date Site Confirmed\nDate of the last contract signature of all the organisations involved (i.e. sponsor, site, 3rd party) or date of final written agreement of statement of activity (as applicable)",date_site_ready_to_start,date_of_first_patient_recruited,site_patient_recruitment,site_recruitment_status,uk_wide_patient_recruitment_target,6.1. Site notes to the ECMC Programme Office,6.2 PO Notes to Sites,6.3 PO Additional Data - TRIAL INTERVENTIONS
0,Belfast,17039SS-SS,Yes,"A Phase I/IIA Study to Assess Safety, Tolerabi...",FAK-PD1,35657,200488,16/WS/0093,2015-003928-31;NCT02758587,Phase I/II,...,28/09/2017,11/10/2017,24/11/2017,26/03/2018,0,"Closed, Follow Up Complete",57,NaN,NaN,"defactinib (Anticancer, other);pembrolizumab (..."
1,Glasgow,SA172,Yes,"A Phase I/IIA Study to Assess Safety, Tolerabi...",FAK-PD1,35657,200488,16/WS/0093,2015-003928-31;NCT02758587,Phase I/II,...,24/04/2017,04/05/2017,23/05/2017,04/07/2017,0,"Closed, in Follow up",57,"2.8 Blood biomarkers, Archival tissue tumour b...",NaN,"defactinib (Anticancer, other);pembrolizumab (..."
2,Southampton,RHM CAN1310,Yes,"A Phase I/IIA Study to Assess Safety, Tolerabi...",FAK-PD1,35657,200488,16/WS/0093,2015-003928-31;NCT02758587,Phase I/II,...,08/09/2017,23/10/2017,12/12/2017,18/01/2018,0,"Closed, Follow Up Complete",57,NaN,NaN,"defactinib (Anticancer, other);pembrolizumab (..."
3,Oxford,13155,Yes,"A Phase 1, Open to recruitment-label, Dose-Fin...",CC-90009,35718,232667,17/SC/0531,2017-001535-39;NCT02848001,Phase I,...,05/10/2018,05/10/2018,13/11/2018,01/10/2018,0,"Closed, in Follow up",6,NaN,NaN,"CC-90009 (Anticancer, other)"
4,Cardiff,NaN,0,Phase 1b/II Trial of Checkpoint Inhibitor (Pem...,CORINTH,35838,228287,19/LO/0314,2017-002300-27,Phase I/II,...,10/09/2021,09/09/2021,10/09/2021,01/11/2021,1,Recruitment paused/on hold,45,NaN,,"pembrolizumab (Monoclonal antibody, humanized)..."


In [10]:
### drop the remaining columns

# Drop multiple columns by passing a list of column names
df.drop(columns=['0.2. Your reference ID\nOPTIONAL\nThis is the ID that you use at your centre to identify the trial.\nThis will not be used by the ECMC Programme office, it is included to help you report.', 
                 '0.3. ECMC Supported?\nPlease select Yes/No to indicate if the trial uses  significan ECMC funded STAFF resources  ',
                 '1.6. Clinical Trial Register ID\nIf the trial is on any of the following clinical Trial Registers, please provide the register ID:\nEU Clinical Trials Register: EUDRACT ID\nClinicalTrials.gov: NCTXXXXXXX\nISRCTN: ISRCTNXXXXX',
                 '1.11. Trial Funder/s',
                 '2.8. Does the trial require RESOURCE-INTENSIVE SCREENING with multiple steps: e.g molecular screening, biopsy, imaging, other -(if other, please provide additional info in Section 6)\n\nMultiple select drop down\nNOTE: if you have selected a disease by error, delete all contents of the cell and re-enter the correct field options',
                 '3.4. Are any researcher/clinician at Site part of the trial steering group\nPlease indicate if a researcher/clinician at your Site is/was part of the trial steering group.\nSteering committees are responsible for decision-making and are the highest level of oversight committee for a clinical trial. \nIf there is no Steering Committee for this trial, please indicate if a researcher/clinician at your site is a key decision maker for this trial',
                 '3.6. Is the UK CI the GLOBAL CI for this trial \nOnly to be completed for international; multi-site studies\nIf the trial is also running internationally and the CI is based at your site, is the Chief Investigator the Global CI for the trial based\n\n(LEAD Site to complete)',
                 '4.2. Date Site Invited \nDate of the email from the Sponsor to the site which provides the protocol in the version to be submitted for regulatory review\nON NIHR Performance in initiating and Delivery of trials list',
                 '4.3. Date Site Selected \nDate on the Sponsor email received by the site providing the minimum defined documents to enable site to commence arrangement and/or confirmation of local capacity and capability as applicable representing that the site has been selected to take part in the trial. ',
                 '**NEW**\n4.4. Date of receipt of HRA document submission pack from sponsors \n(aka Document submission pack)\nDate of receipt of all necessary documentation required for the permissions and approvals process.',
                 '4.6. Date Site Confirmed By Sponsor\nDate of the first contract signature of all the organisations involved (i.e. sponsor, site, 3rd party) or Date on the email received from Sponsor providing the final statement of activity ready for final agreement. ',
                 '4.7. Date Site Confirmed\nDate of the last contract signature of all the organisations involved (i.e. sponsor, site, 3rd party) or date of final written agreement of statement of activity (as applicable)',
                 '6.1. Site notes to the ECMC Programme Office',
                 '6.2 PO Notes to Sites',
                 '6.3 PO Additional Data - TRIAL INTERVENTIONS',             
                ], inplace=True)

# Display the updated DataFrame
df.head()

,ecmc_name,full_trial_title,trial_acronym,cpms_id,iras_id,rec_reference_number,trial_phase,cancer_type,treatment_type,sponsor,...,total_accumulative_number_of_experimental_arms,number_of_simultaneously_open_experimental_arms,lead_site,date_of_hra_application,hra_approval_date,date_site_ready_to_start,date_of_first_patient_recruited,site_patient_recruitment,site_recruitment_status,uk_wide_patient_recruitment_target
0,Belfast,"A Phase I/IIA Study to Assess Safety, Tolerabi...",FAK-PD1,35657,200488,16/WS/0093,Phase I/II,Non-small cell lung cancer; Pancreatic;Mesothe...,Combination study - Drug/radiotherapy/surgery/...,NHS Greater Glasgow & Clyde,...,2,2,No,NaN,NaN,24/11/2017,26/03/2018,0,"Closed, Follow Up Complete",57
1,Glasgow,"A Phase I/IIA Study to Assess Safety, Tolerabi...",FAK-PD1,35657,200488,16/WS/0093,Phase I/II,Non-small cell lung cancer; Pancreatic;Mesothe...,Combination study - Drug/radiotherapy/surgery/...,NHS Greater Glasgow & Clyde,...,2,2,No,07/04/2017,25/04/2017,23/05/2017,04/07/2017,0,"Closed, in Follow up",57
2,Southampton,"A Phase I/IIA Study to Assess Safety, Tolerabi...",FAK-PD1,35657,200488,16/WS/0093,Phase I/II,Non-small cell lung cancer; Pancreatic;Mesothe...,Combination study - Drug/radiotherapy/surgery/...,NHS Greater Glasgow & Clyde,...,2,2,No,NaN,25/04/2017,12/12/2017,18/01/2018,0,"Closed, Follow Up Complete",57
3,Oxford,"A Phase 1, Open to recruitment-label, Dose-Fin...",CC-90009,35718,232667,17/SC/0531,Phase I,Myelodysplastic Syndromes (MDS);Acute myeloid ...,Other,Bristol-Myers Squibb,...,2,1,Yes,15/09/2017,18/12/2017,13/11/2018,01/10/2018,0,"Closed, in Follow up",6
4,Cardiff,Phase 1b/II Trial of Checkpoint Inhibitor (Pem...,CORINTH,35838,228287,19/LO/0314,Phase I/II,Colorectal,Combination study - Drug/radiotherapy/surgery/...,Cardiff University,...,1,1,No,NaN,NaN,10/09/2021,01/11/2021,1,Recruitment paused/on hold,45


In [11]:
print(df.dtypes)

ecmc_name                                          object
full_trial_title                                   object
trial_acronym                                      object
cpms_id                                             int64
iras_id                                             int64
rec_reference_number                               object
trial_phase                                        object
cancer_type                                        object
treatment_type                                     object
sponsor                                            object
investigator_initiated_study                       object
international_national_trial                       object
patient_age_group                                  object
dose_escalation                                    object
first_in_class                                     object
first_in_child                                     object
precision_medicine                                 object
complex_innova

In [12]:
##change data types
##covert integer columns to integers

df['site_patient_recruitment'] = df['site_patient_recruitment'].astype('Int64')
df['iras_id'] = df['iras_id'].astype('Int64')

# Convert date columns to datetime
df['date_of_hra_application'] = pd.to_datetime(df['date_of_hra_application'], dayfirst=True)
df['hra_approval_date'] = pd.to_datetime(df['hra_approval_date'], dayfirst=True)
df['date_site_ready_to_start'] = pd.to_datetime(df['date_site_ready_to_start'], dayfirst=True)
df['date_of_first_patient_recruited'] = pd.to_datetime(df['date_of_first_patient_recruited'], dayfirst=True)


In [13]:
##test that the data is in the right format in each column type

### check that the data types of specific columns are correct

try:
    # Expected data types for specific columns
    expected_dtypes = {
        
        'site_patient_recruitment' : 'Int64',
        'iras_id' : 'Int64',
        'hra_approval_date': 'datetime64[ns]',
        'date_of_hra_application' : 'datetime64[ns]',
        'date_site_ready_to_start' : 'datetime64[ns]',
        'date_of_first_patient_recruited' : 'datetime64[ns]'

    }

 # Loop through each column in expected_dtypes and compare with actual data type
    for col, expected_dtype in expected_dtypes.items():
        actual_dtype = df[col].dtype
        assert str(actual_dtype) == expected_dtype, f"Expected {expected_dtype} for column '{col}', but got {actual_dtype}"

    logging.info('Column Data Types: OK')

except KeyError as e:
    logging.error(f"ERROR: Column not found: {e}")
    sys.exit(1)

except AssertionError as e:
    logging.error(f"ERROR: Data type mismatch: {e}")
    sys.exit(1)

except Exception as e:
    logging.error(f"ERROR: An unexpected error occurred: {e}")
    sys.exit(1)

In [14]:
##no nulls in the columns that are formatted as "NOT NULL"

# List of columns to check for non-null values
columns_to_check = ['ecmc_name', 'iras_id', 'site_recruitment_status','trial_phase','full_trial_title']

try:
    for column in columns_to_check:
        # Check if there are any null values in the column
        if df[column].isnull().any():
            raise AssertionError(f"Column '{column}' contains null values.")
    
    logging.info('All specified columns are non-null.')

except KeyError as e:
    logging.error(f"ERROR: Column not found: {e}")
    sys.exit(1)

except AssertionError as e:
    logging.error(f"ERROR: {e}")
    sys.exit(1)

except Exception as e:
    logging.error(f"ERROR: An unexpected error occurred: {e}")
    sys.exit(1)


In [15]:
## test that there are no duplicate submission entries (Centre & IRAS together)

try:
    # Test to check for duplicates based on the combination of ECMC_name and IRAS_ID
    if df.duplicated(subset=['ecmc_name', 'iras_id']).any():
        raise AssertionError("There are duplicates based on the combination of 'ECMC_name' and 'iras_id'.")

    logging.info('No duplicates found based on the combination of ECMC_name and iras_id.')

except KeyError as e:
    logging.error(f"ERROR: Column not found: {e}")
    sys.exit(1)

except AssertionError as e:
    logging.error(f"ERROR: {e}")
    sys.exit(1)

except Exception as e:
    logging.error(f"ERROR: An unexpected error occurred: {e}")
    sys.exit(1)


In [16]:
### add a year column - CHANGES EACH YEAR

# Add a new column with a constant value
df['Submission_Year'] = '2022-2023'

# Display the updated DataFrame
df.head()


,ecmc_name,full_trial_title,trial_acronym,cpms_id,iras_id,rec_reference_number,trial_phase,cancer_type,treatment_type,sponsor,...,number_of_simultaneously_open_experimental_arms,lead_site,date_of_hra_application,hra_approval_date,date_site_ready_to_start,date_of_first_patient_recruited,site_patient_recruitment,site_recruitment_status,uk_wide_patient_recruitment_target,Submission_Year
0,Belfast,"A Phase I/IIA Study to Assess Safety, Tolerabi...",FAK-PD1,35657,200488,16/WS/0093,Phase I/II,Non-small cell lung cancer; Pancreatic;Mesothe...,Combination study - Drug/radiotherapy/surgery/...,NHS Greater Glasgow & Clyde,...,2,No,NaT,NaT,2017-11-24,2018-03-26,0,"Closed, Follow Up Complete",57,2022-2023
1,Glasgow,"A Phase I/IIA Study to Assess Safety, Tolerabi...",FAK-PD1,35657,200488,16/WS/0093,Phase I/II,Non-small cell lung cancer; Pancreatic;Mesothe...,Combination study - Drug/radiotherapy/surgery/...,NHS Greater Glasgow & Clyde,...,2,No,2017-04-07,2017-04-25,2017-05-23,2017-07-04,0,"Closed, in Follow up",57,2022-2023
2,Southampton,"A Phase I/IIA Study to Assess Safety, Tolerabi...",FAK-PD1,35657,200488,16/WS/0093,Phase I/II,Non-small cell lung cancer; Pancreatic;Mesothe...,Combination study - Drug/radiotherapy/surgery/...,NHS Greater Glasgow & Clyde,...,2,No,NaT,2017-04-25,2017-12-12,2018-01-18,0,"Closed, Follow Up Complete",57,2022-2023
3,Oxford,"A Phase 1, Open to recruitment-label, Dose-Fin...",CC-90009,35718,232667,17/SC/0531,Phase I,Myelodysplastic Syndromes (MDS);Acute myeloid ...,Other,Bristol-Myers Squibb,...,1,Yes,2017-09-15,2017-12-18,2018-11-13,2018-10-01,0,"Closed, in Follow up",6,2022-2023
4,Cardiff,Phase 1b/II Trial of Checkpoint Inhibitor (Pem...,CORINTH,35838,228287,19/LO/0314,Phase I/II,Colorectal,Combination study - Drug/radiotherapy/surgery/...,Cardiff University,...,1,No,NaT,NaT,2021-09-10,2021-11-01,1,Recruitment paused/on hold,45,2022-2023


In [17]:
##Split tables into the tables required

table1_columns = ['ecmc_name', 'iras_id', 'number_of_patients_screened', 'total_accumulative_number_of_experimental_arms', 'number_of_simultaneously_open_experimental_arms','lead_site', 'date_of_hra_application', 'hra_approval_date', 'date_site_ready_to_start', 'date_of_first_patient_recruited', 'site_patient_recruitment', 'site_recruitment_status', 'Submission_Year']

centre_submissions = df[table1_columns]

centre_submissions.head()



,ecmc_name,iras_id,number_of_patients_screened,total_accumulative_number_of_experimental_arms,number_of_simultaneously_open_experimental_arms,lead_site,date_of_hra_application,hra_approval_date,date_site_ready_to_start,date_of_first_patient_recruited,site_patient_recruitment,site_recruitment_status,Submission_Year
0,Belfast,200488,0,2,2,No,NaT,NaT,2017-11-24,2018-03-26,0,"Closed, Follow Up Complete",2022-2023
1,Glasgow,200488,0,2,2,No,2017-04-07,2017-04-25,2017-05-23,2017-07-04,0,"Closed, in Follow up",2022-2023
2,Southampton,200488,0,2,2,No,NaT,2017-04-25,2017-12-12,2018-01-18,0,"Closed, Follow Up Complete",2022-2023
3,Oxford,232667,0,2,1,Yes,2017-09-15,2017-12-18,2018-11-13,2018-10-01,0,"Closed, in Follow up",2022-2023
4,Cardiff,228287,0,1,1,No,NaT,NaT,2021-09-10,2021-11-01,1,Recruitment paused/on hold,2022-2023


In [18]:
### test that the ecmc names are the right values

try:
    # Predefined valid values (dropdown options) for 'site_recruitment_status_id'
    valid_values = {
        'Barts', 'Belfast', 'Birmingham', 'Birmingham_PAED' , 'Bristol_PAED', 'Cambridge', 'Cambridge_PAED', 'Cardiff', 'Cardiff_PAED', 'Edinburgh', 'Glasgow', 'Glasgow_PAED', 'GOSH_PAED', 'ICR', 'Imperial', 'KHP', 'Leeds_PAED', 'Leicester', 'Liverpool', 'Liverpool_PAED', 'Manchester', 'Manchester_PAED', 'Newcastle', 'Newcastle_PAED', 'Oxford', 'Royal Marsden_PAED', 'Southampton', 'Southampton_PAED', 'UCL' 
    }

    # Column to validate
    column_to_check = 'ecmc_name'

    # Check if the column exists
    if column_to_check not in df.columns:
        raise KeyError(f"Column '{column_to_check}' not found in the DataFrame")

    # Get the unique values in the column
    column_values = df[column_to_check].unique()

    # Check if all values in the column are in the predefined valid values
    invalid_values = set(column_values) - valid_values

    # Assert there are no invalid values
    assert not invalid_values, f"Found invalid values in '{column_to_check}': {invalid_values}"

    logging.info(f"Column '{column_to_check}' values: OK")

except KeyError as e:
    logging.error(f"ERROR: Column not found: {e}")
    sys.exit(1)

except AssertionError as e:
    logging.error(f"ERROR: Invalid values found: {e}")
    sys.exit(1)

except Exception as e:
    logging.error(f"ERROR: An unexpected error occurred: {e}")
    sys.exit(1)

In [19]:
# Get the ECMC_ID in to the table

ECMC_mapping = {
    'Barts' : 1,
    'Belfast' : 2,
    'Birmingham' : 3,
    'Birmingham_PAED' : 4,
    'Bristol_PAED' : 5,
    'Cambridge' : 6,
    'Cambridge_PAED' : 7,
    'Cardiff' : 8,
    'Cardiff_PAED' : 9,
    'Edinburgh' : 10,
    'Glasgow' : 11,
    'Glasgow_PAED' : 12,
    'GOSH_PAED' : 13,
    'ICR' : 14,
    'Imperial' : 15,
    'KHP' : 16,
    'Leeds_PAED' : 17,
    'Leicester' : 18,
    'Liverpool' : 19,
    'Liverpool_PAED' : 20,
    'Manchester' : 21,
    'Manchester_PAED' : 22,
    'Newcastle' : 23,
    'Newcastle_PAED' : 24,
    'Oxford' : 25,
    'Royal Marsden_PAED' : 26,
    'Southampton' : 27,
    'Southampton_PAED' : 28,
    'UCL' : 29,
}

## convert into datafram

# Convert the dictionary to a DataFrame for easier merging
ECMC_ID_df = pd.DataFrame(list(ECMC_mapping.items()), columns=['ecmc_name', 'ecmc_id'])

# Display the manually created site DataFrame
ECMC_ID_df.head()

,ecmc_name,ecmc_id
0,Barts,1
1,Belfast,2
2,Birmingham,3
3,Birmingham_PAED,4
4,Bristol_PAED,5


In [20]:
### In the new centre_submissions table, replace the ECMC_name with the ECMC_ID

# Merge the original DataFrame (df) with the manually created sites DataFrame (sites_df)
centre_submissions_v1 = centre_submissions.merge(ECMC_ID_df, on='ecmc_name')

# Drop the 'ECMC_name' column 
centre_submissions_v1.drop(columns=['ecmc_name'], inplace=True)

# Display the updated DataFrame with site_id
centre_submissions_v1

,iras_id,number_of_patients_screened,total_accumulative_number_of_experimental_arms,number_of_simultaneously_open_experimental_arms,lead_site,date_of_hra_application,hra_approval_date,date_site_ready_to_start,date_of_first_patient_recruited,site_patient_recruitment,site_recruitment_status,Submission_Year,ecmc_id
0,200488,0,2,2,No,NaT,NaT,2017-11-24,2018-03-26,0,"Closed, Follow Up Complete",2022-2023,2
1,200488,0,2,2,No,2017-04-07,2017-04-25,2017-05-23,2017-07-04,0,"Closed, in Follow up",2022-2023,11
2,200488,0,2,2,No,NaT,2017-04-25,2017-12-12,2018-01-18,0,"Closed, Follow Up Complete",2022-2023,27
3,232667,0,2,1,Yes,2017-09-15,2017-12-18,2018-11-13,2018-10-01,0,"Closed, in Follow up",2022-2023,25
4,228287,0,1,1,No,NaT,NaT,2021-09-10,2021-11-01,1,Recruitment paused/on hold,2022-2023,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,237080,1,3,2,No,NaT,2019-07-23,2021-06-28,2021-07-14,1,"Closed, in Follow up",2022-2023,13
61,235714,0,2,1,Yes,2017-12-20,2018-03-08,2018-04-27,2018-10-01,0,"Closed, in Follow up",2022-2023,25
62,230866,0,2,2,Yes,NaT,NaT,2018-01-24,2018-02-08,0,"Closed, in Follow up",2022-2023,1
63,233362,0,8,3,Yes,2017-12-08,2018-02-08,2018-07-27,2018-08-07,11,Open to Recruitment,2022-2023,1


In [21]:
### test that the recruitment statuses have the right values

try:
    # Predefined valid values (dropdown options) for 'site_recruitment_status_id'
    valid_values = {
        'Closed, in Follow up', 'Recruitment paused/on hold', 'Open to Recruitment', 'Closed, Follow Up Complete' , 'Withdrawn', 'Suspended', 'In Setup', 'Early termination'
    }

    # Column to validate
    column_to_check_1 = 'site_recruitment_status'

    # Check if the column exists
    if column_to_check_1 not in df.columns:
        raise KeyError(f"Column '{column_to_check_1}' not found in the DataFrame")

    # Get the unique values in the column
    column_values = df[column_to_check_1].unique()

    # Check if all values in the column are in the predefined valid values
    invalid_values = set(column_values) - valid_values

    # Assert there are no invalid values
    assert not invalid_values, f"Found invalid values in '{column_to_check_1}': {invalid_values}"

    logging.info(f"Column '{column_to_check_1}' values: OK")

except KeyError as e:
    logging.error(f"ERROR: Column not found: {e}")
    sys.exit(1)

except AssertionError as e:
    logging.error(f"ERROR: Invalid values found: {e}")
    sys.exit(1)

except Exception as e:
    logging.error(f"ERROR: An unexpected error occurred: {e}")
    sys.exit(1)

In [22]:
# Get the SITE_Recruitment_Status_ID in to the table

site_recruitment_status_mapping = {
    'Closed, in Follow up' : 1,
    'Recruitment paused/on hold' : 2,
    'Open to Recruitment' : 3,
    'Closed, Follow Up Complete' : 4,
    'Withdrawn' : 5,
    'Suspended' : 6,
    'In Setup' : 7,
    'Early termination' : 8,
}

## convert into dataframe

site_recruitment_status_df = pd.DataFrame(list(site_recruitment_status_mapping.items()), columns=['site_recruitment_status','site_recruitment_status_id'])

# Display the manually created site DataFrame
site_recruitment_status_df.head()

,site_recruitment_status,site_recruitment_status_id
0,"Closed, in Follow up",1
1,Recruitment paused/on hold,2
2,Open to Recruitment,3
3,"Closed, Follow Up Complete",4
4,Withdrawn,5


In [23]:
##replace recruitment status with recruitment status id

# Merge the original DataFrame (df) with the manually created sites DataFrame (sites_df)
centre_submissions_v2 = centre_submissions_v1.merge(site_recruitment_status_df, on='site_recruitment_status')

# Drop the 'site_recruitment_status' column 
centre_submissions_v2.drop(columns=['site_recruitment_status'], inplace=True)

# Display the updated DataFrame with site_id
centre_submissions_v2.head()

,iras_id,number_of_patients_screened,total_accumulative_number_of_experimental_arms,number_of_simultaneously_open_experimental_arms,lead_site,date_of_hra_application,hra_approval_date,date_site_ready_to_start,date_of_first_patient_recruited,site_patient_recruitment,Submission_Year,ecmc_id,site_recruitment_status_id
0,200488,0,2,2,No,NaT,NaT,2017-11-24,2018-03-26,0,2022-2023,2,4
1,200488,0,2,2,No,2017-04-07,2017-04-25,2017-05-23,2017-07-04,0,2022-2023,11,1
2,200488,0,2,2,No,NaT,2017-04-25,2017-12-12,2018-01-18,0,2022-2023,27,4
3,232667,0,2,1,Yes,2017-09-15,2017-12-18,2018-11-13,2018-10-01,0,2022-2023,25,1
4,228287,0,1,1,No,NaT,NaT,2021-09-10,2021-11-01,1,2022-2023,8,2


In [24]:
##Split tables into the tables required

table2_columns = ['iras_id', 'full_trial_title','trial_acronym', 'cpms_id', 'rec_reference_number', 'trial_phase', 'investigator_initiated_study', 'international_national_trial', 'patient_age_group', 'dose_escalation', 'first_in_class', 'first_in_child', 'precision_medicine', 'complex_innovative_design', 'randomised', 'pre_screening_processes', 'uk_wide_patient_recruitment_target']

individual_clinical_trials = df[table2_columns].drop_duplicates(subset='iras_id')

individual_clinical_trials.head()

,iras_id,full_trial_title,trial_acronym,cpms_id,rec_reference_number,trial_phase,investigator_initiated_study,international_national_trial,patient_age_group,dose_escalation,first_in_class,first_in_child,precision_medicine,complex_innovative_design,randomised,pre_screening_processes,uk_wide_patient_recruitment_target
0,200488,"A Phase I/IIA Study to Assess Safety, Tolerabi...",FAK-PD1,35657,16/WS/0093,Phase I/II,Yes,UK Multisite,adult (18+),No,No,NaN,No,No,Yes,N/A,57
3,232667,"A Phase 1, Open to recruitment-label, Dose-Fin...",CC-90009,35718,17/SC/0531,Phase I,No,International,adult (18+),Yes,Yes,No,No,No,No,N/A,6
4,228287,Phase 1b/II Trial of Checkpoint Inhibitor (Pem...,CORINTH,35838,19/LO/0314,Phase I/II,Yes,UK Multisite,adult (18+),Yes,No,No,No,No,No,N/A,45
6,226255,"A Single-Arm, Open-Label, Multi-Centre, Phase ...",ALEXANDER,35925,17/LO/0812,Phase I/II,No,International,adult (18+),Yes,No,No,No,Yes,No,None,NaN
7,225064,FRAME: A Phase I Trial of the Combination of V...,FRAME,35942,17/LO/1473,Phase I,Yes,UK Multisite,adult (18+),Unknown,Unknown,N/A,Yes,Yes,No,yes,110


In [25]:
# Get the Trial_Phase_ID in to the table

trial_phase_mapping = {
    'Phase 0' : 1,
    'Phase I' : 2,
    'Phase I/Ib' : 3,
    'Phase Ia/Ib' : 4,
    'Phase II' : 5,
    'Phase I/IIa' : 6,
    'Phase Ib/II' : 7,
    'Phase I/II' : 8,
    'Phase II/III' : 9,
    'Phase III' : 10,
    'N/A' : 11,
    'N/A - molecular profiling for trial eligibility' : 12,
    'Not Applicable' : 13,
    'N/A - surgical' : 14,
    'N/A - imaging' : 15,
    'Pilot/Feasibility' : 16,
    'Molecular profiling' : 17,
    'Other' : 18,
}

## convert into dataframe

trial_phase_df = pd.DataFrame(list(trial_phase_mapping.items()), columns=['trial_phase','trial_phase_id'])

# Display the manually created site DataFrame
trial_phase_df.head()

,trial_phase,trial_phase_id
0,Phase 0,1
1,Phase I,2
2,Phase I/Ib,3
3,Phase Ia/Ib,4
4,Phase II,5


In [26]:
##replace trial phase with ID in individual trials

# Merge the original DataFrame (df) with the manually created sites DataFrame (sites_df)
individual_clinical_trials = individual_clinical_trials.merge(trial_phase_df, on='trial_phase')

# Drop the 'site_recruitment_status' column 
individual_clinical_trials.drop(columns=['trial_phase'], inplace=True)

# Display the updated DataFrame with site_id
individual_clinical_trials.head()

,iras_id,full_trial_title,trial_acronym,cpms_id,rec_reference_number,investigator_initiated_study,international_national_trial,patient_age_group,dose_escalation,first_in_class,first_in_child,precision_medicine,complex_innovative_design,randomised,pre_screening_processes,uk_wide_patient_recruitment_target,trial_phase_id
0,200488,"A Phase I/IIA Study to Assess Safety, Tolerabi...",FAK-PD1,35657,16/WS/0093,Yes,UK Multisite,adult (18+),No,No,NaN,No,No,Yes,N/A,57,8
1,232667,"A Phase 1, Open to recruitment-label, Dose-Fin...",CC-90009,35718,17/SC/0531,No,International,adult (18+),Yes,Yes,No,No,No,No,N/A,6,2
2,228287,Phase 1b/II Trial of Checkpoint Inhibitor (Pem...,CORINTH,35838,19/LO/0314,Yes,UK Multisite,adult (18+),Yes,No,No,No,No,No,N/A,45,8
3,226255,"A Single-Arm, Open-Label, Multi-Centre, Phase ...",ALEXANDER,35925,17/LO/0812,No,International,adult (18+),Yes,No,No,No,Yes,No,None,NaN,8
4,225064,FRAME: A Phase I Trial of the Combination of V...,FRAME,35942,17/LO/1473,Yes,UK Multisite,adult (18+),Unknown,Unknown,N/A,Yes,Yes,No,yes,110,2


In [27]:
# Pull existing centre submissions from postgres
specified_table = 'centre_submissions'

# Extract the specified table
existing_centre_submissions = extractPostgresTable(engine, specified_table)

# Display the first few rows
existing_centre_submissions.head()

,centre_submission_id,ecmc_id,iras_id,number_of_patients_screened,total_accumulative_number_of_experimental_arms,number_of_simultaneously_open_experimental_arms,lead_site,date_of_hra_application,hra_approval_date,date_site_ready_to_start,date_of_first_patient_recruited,site_patient_recruitment,site_recruitment_status_id,Submission_Year
0,1510,12,73862,N/A,3,3,No,None,None,2011-12-05,None,0.0,4,2022-2023
1,354,29,245336,N/A,None,None,No,None,None,None,None,NaN,7,2022-2023
2,1511,2,200488,0,2,2,No,None,None,2017-11-24,2018-03-26,0.0,4,2022-2023
3,1512,11,200488,0,2,2,No,2017-04-07,2017-04-25,2017-05-23,2017-07-04,0.0,1,2022-2023
4,1513,27,200488,0,2,2,No,None,2017-04-25,2017-12-12,2018-01-18,0.0,4,2022-2023


In [28]:
# Get the maximum Centre_submission_id from existing data
max_existing_id = existing_centre_submissions['centre_submission_id'].max()

print("Max existing Centre_submission_id:", max_existing_id)

Max existing Centre_submission_id: 1562


In [29]:
# Assign new Centre_submission_id to new records in centre_submissions
# We create a range starting from max_existing_id + 1
new_ids_start = max_existing_id + 1
new_ids_end = new_ids_start + len(centre_submissions_v2)

# Assign these new IDs to the 'Centre_submission_id' column in centre_submissions
centre_submissions_v2['centre_submission_id'] = range(new_ids_start, new_ids_end)

# Display the updated DataFrame with new Centre_submission_id
centre_submissions_v2.head()

,iras_id,number_of_patients_screened,total_accumulative_number_of_experimental_arms,number_of_simultaneously_open_experimental_arms,lead_site,date_of_hra_application,hra_approval_date,date_site_ready_to_start,date_of_first_patient_recruited,site_patient_recruitment,Submission_Year,ecmc_id,site_recruitment_status_id,centre_submission_id
0,200488,0,2,2,No,NaT,NaT,2017-11-24,2018-03-26,0,2022-2023,2,4,1563
1,200488,0,2,2,No,2017-04-07,2017-04-25,2017-05-23,2017-07-04,0,2022-2023,11,1,1564
2,200488,0,2,2,No,NaT,2017-04-25,2017-12-12,2018-01-18,0,2022-2023,27,4,1565
3,232667,0,2,1,Yes,2017-09-15,2017-12-18,2018-11-13,2018-10-01,0,2022-2023,25,1,1566
4,228287,0,1,1,No,NaT,NaT,2021-09-10,2021-11-01,1,2022-2023,8,2,1567


In [30]:
# Desired column order
new_column_order = ['centre_submission_id', 'ecmc_id', 'iras_id', 'number_of_patients_screened','total_accumulative_number_of_experimental_arms','number_of_simultaneously_open_experimental_arms','lead_site','date_of_hra_application','hra_approval_date','date_site_ready_to_start','date_of_first_patient_recruited','site_patient_recruitment','site_recruitment_status_id','Submission_Year']

# Reorder the DataFrame columns
centre_submissions_reordered = centre_submissions_v2[new_column_order]

centre_submissions_reordered

,centre_submission_id,ecmc_id,iras_id,number_of_patients_screened,total_accumulative_number_of_experimental_arms,number_of_simultaneously_open_experimental_arms,lead_site,date_of_hra_application,hra_approval_date,date_site_ready_to_start,date_of_first_patient_recruited,site_patient_recruitment,site_recruitment_status_id,Submission_Year
0,1563,2,200488,0,2,2,No,NaT,NaT,2017-11-24,2018-03-26,0,4,2022-2023
1,1564,11,200488,0,2,2,No,2017-04-07,2017-04-25,2017-05-23,2017-07-04,0,1,2022-2023
2,1565,27,200488,0,2,2,No,NaT,2017-04-25,2017-12-12,2018-01-18,0,4,2022-2023
3,1566,25,232667,0,2,1,Yes,2017-09-15,2017-12-18,2018-11-13,2018-10-01,0,1,2022-2023
4,1567,8,228287,0,1,1,No,NaT,NaT,2021-09-10,2021-11-01,1,2,2022-2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,1623,13,237080,1,3,2,No,NaT,2019-07-23,2021-06-28,2021-07-14,1,1,2022-2023
61,1624,25,235714,0,2,1,Yes,2017-12-20,2018-03-08,2018-04-27,2018-10-01,0,1,2022-2023
62,1625,1,230866,0,2,2,Yes,NaT,NaT,2018-01-24,2018-02-08,0,1,2022-2023
63,1626,1,233362,0,8,3,Yes,2017-12-08,2018-02-08,2018-07-27,2018-08-07,11,3,2022-2023


In [31]:
# Pull existing centre submissions from postgres
specified_table = 'individual_clinical_trials'

# Extract the specified table
existing_individual_clinical_trials = extractPostgresTable(engine, specified_table)

# Display the first few rows
existing_individual_clinical_trials.head()

,iras_id,full_trial_title,trial_acronym,cpms_id,rec_reference_number,trial_phase_id,investigator_initiated_study,international_national_trial,patient_age_group,dose_escalation,first_in_class,first_in_child,precision_medicine,complex_innovative_design,randomised,pre_screening_processes,uk_wide_patient_recruitment_target
0,21769,Transanal Endoscopic Microsurgery (TEM) and Ra...,N/A,None,10/H1202/81,5,No,Unknown,18 +,No,No,N/A,No,No,Yes,N/A,NaN
1,23167,PulMiCC: To determine the feasibility of recru...,N/A,9018,10/H0720/5,10,Yes,International,18 +,No,No,N/A,No,No,Yes,N/A,NaN
2,28730,A Phase I Clinical Study to Determine the Opti...,ACIT-1,None,13/NW/0291,8,No,UK Multisite,18+,Yes,No,No,No,No,No,N/A,34.0
3,35212,An Open-Label Multicenter Phase 1 Study of E73...,PENT 101,35212,17/LO/0660,2,No,International,18+,Yes,No,N/A,Yes,Yes,No,No,NaN
4,57563,Evaluation of a Non-Endoscopic Immunocytologic...,Evaluation of a NonEndoscopic Device for Barre...,9461,10/H0308/71,11,Yes,UK Multisite,18+,No,No,No,No,No,No,N/A,1850.0


In [32]:
#### remove existing records from table to post to postgres

existing_iras_ids = existing_individual_clinical_trials['iras_id']

new_individual_trials = individual_clinical_trials[~individual_clinical_trials['iras_id'].isin(existing_iras_ids)]

new_individual_trials.head()

,iras_id,full_trial_title,trial_acronym,cpms_id,rec_reference_number,investigator_initiated_study,international_national_trial,patient_age_group,dose_escalation,first_in_class,first_in_child,precision_medicine,complex_innovative_design,randomised,pre_screening_processes,uk_wide_patient_recruitment_target,trial_phase_id
16,230053,"A Phase 1, Open-Label, Multicentre, Non-Random...",Study of AZD4573 in Relapsed or Refractory Hae...,35267,17/EE/0317,No,UK Multisite,Adult (18+),Unknown,Unknown,No,Unknown,Unknown,No,NaN,NaN,2
17,229775,A Phase 1b Trial of Hu5F9-G4 monotherapy or Hu...,5F9005,35296,17/SC/0450,No,International,adult (18+),Yes,No,No,Yes,No,No,N/A,66,2
27,235018,Phase I/II a Dose-Escalation and Expansion Stu...,Study to evaluate CORT125281 with Enzalutamide...,36145,17/LO/2032,No,International,adult (18+),Yes,No,NaN,No,No,Yes,N/A,32,8
29,237080,A Phase I Study of the Safety and Pharmacokine...,M13-833,36224,19/NE/0130,No,International,"child (0-18), adult (18+)",No,No,No,No,Yes,No,N/A,UNK,2


In [33]:
# Desired column order
new_column_order_1 = ['iras_id', 'full_trial_title', 'trial_acronym','cpms_id','rec_reference_number','trial_phase_id','investigator_initiated_study','international_national_trial','patient_age_group','dose_escalation','first_in_class','first_in_child','precision_medicine','complex_innovative_design','randomised','pre_screening_processes','uk_wide_patient_recruitment_target']

# Reorder the DataFrame columns
individual_clinical_trials_reordered = new_individual_trials[new_column_order_1]

individual_clinical_trials_reordered.head()

,iras_id,full_trial_title,trial_acronym,cpms_id,rec_reference_number,trial_phase_id,investigator_initiated_study,international_national_trial,patient_age_group,dose_escalation,first_in_class,first_in_child,precision_medicine,complex_innovative_design,randomised,pre_screening_processes,uk_wide_patient_recruitment_target
16,230053,"A Phase 1, Open-Label, Multicentre, Non-Random...",Study of AZD4573 in Relapsed or Refractory Hae...,35267,17/EE/0317,2,No,UK Multisite,Adult (18+),Unknown,Unknown,No,Unknown,Unknown,No,NaN,NaN
17,229775,A Phase 1b Trial of Hu5F9-G4 monotherapy or Hu...,5F9005,35296,17/SC/0450,2,No,International,adult (18+),Yes,No,No,Yes,No,No,N/A,66
27,235018,Phase I/II a Dose-Escalation and Expansion Stu...,Study to evaluate CORT125281 with Enzalutamide...,36145,17/LO/2032,8,No,International,adult (18+),Yes,No,NaN,No,No,Yes,N/A,32
29,237080,A Phase I Study of the Safety and Pharmacokine...,M13-833,36224,19/NE/0130,2,No,International,"child (0-18), adult (18+)",No,No,No,No,Yes,No,N/A,UNK


In [34]:
#### ADD TO POSTGRES

In [35]:
#load into Postgres
#logging.info("Attempting to write to database...")
    
loadToPostgresIncrement(individual_clinical_trials_reordered,engine,"individual_clinical_trials")
logging.info("Writing to table individual_clinical_trials: Complete")



In [36]:
loadToPostgresIncrement(centre_submissions_reordered,engine,"centre_submissions")
logging.info("Writing to table centre_submissions: Complete")